# Place for experimenting the progressive design

In [4]:
import yaml
import inspect
import importlib
import os,sys

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
# from model_discovery.evolution import BuildEvolution
import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code

ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)

prompts_dir='../model_discovery/agents/prompts/'
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))
GAU_TEMPLATE = U.read_file(U.pjoin(prompts_dir,'gau_template.py'))
GAU_BASE=inspect.getsource(GAUBase)


c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


## Parsers

In [55]:
import json
import os,sys
import importlib

sys.path.append('..')

import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code
import model_discovery.agents.prompts.prompts as P
import model_discovery.model.composer
importlib.reload(model_discovery.model.composer)
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
import model_discovery.utils
importlib.reload(model_discovery.utils)
import model_discovery.utils as U



code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn

from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #
from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #
from model_discovery.model.utils.modules import gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #

import torch.nn.functional as F

# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #

# Placeholder classes for future implementation
class MemoryAccessUnit(nn.Module):
    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class DownsamplingUnit(nn.Module):
    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class GAU(GAUBase): # DO NOT CHANGE THE NAME OF THIS CLASS
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable
    """
    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #
        # argv: list of hyperparameters
        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers
        super().__init__(embed_dim) # DO NOT CHANGE THIS LINE #

        self.a1 = kwarg_all.get('a1', 1)
        self.a2: int = kwarg_all.get('a2', 2)

        # COMPLETING THE CODE HERE #
        self.token_scorer = TokenScoringGAU(embed_dim, **factory_kwargs)
        self.dual_path: GAUBase = DualPathGAU(embed_dim, **factory_kwargs)
        self.latent_attention: GAUBase = LatentAttentionGAU(embed_dim, **factory_kwargs)

        self.stages: nn.ModuleList = nn.ModuleList([CompressionStageGAU(embed_dim, device=
            device, dtype=dtype, **kwargs) for _ in range(2)])

        self.model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU(),
            'gau3': CompressionStageGAU()
        })

        self.model2 = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('gau2', DualPathGAU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

    # YOU CAN ADD MORE FUNCTIONS HERE #


    def _forward(self, X, **Z): 

        # THE CODE HERE MUST BE COMPLETED #
        # Step 1: Score tokens
        X, Z = self.token_scorer(X, **Z)
        # Step 2: Route through dual paths
        # Step 3: Apply latent attention
        Y, Z = self.latent_attention(X, **Z)
        X, Z = self.dual_path(X, **Z)

        return Y, Z

@gau_test
def test1(a=1,device=None):
    print('test1')

    
@gau_test
def test2(device=None):
    print('test2')

'''
children = ['TokenScoringGAU', 'DualPathGAU', 'LatentAttentionGAU','CompressionStageGAU'] # solve redundancy 

In [63]:
code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn
import torch.nn.functional as F

from model_discovery.model.utils.modules import GAUBase, gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #


# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #


class RelevanceScoringGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT KEEP THE ORIGINAL ONES #
        self.factory_kwargs = {"device": device, "dtype": dtype} # DO NOT CHANGE THIS LINE, REMEMBER TO PASS IT #
        super().__init__(embed_dim, block_loc, kwarg_all) # DO NOT CHANGE THIS LINE #

        # Initialize a linear projection layer
        self.projection = nn.Linear(embed_dim, embed_dim // 2, **self.factory_kwargs)
        self.projection.weight.requires_grad = True
        self.projection.bias.requires_grad = True

        # Initialize a learnable query vector
        self.query_vector = nn.Parameter(torch.randn(embed_dim // 2, requires_grad=True, **self.factory_kwargs))

    def _forward(self, X, **Z): 
        # Project the input embeddings
        projected_X = self.projection(X)

        # Compute similarity scores
        similarity_scores = torch.matmul(projected_X, self.query_vector)

        # Normalize the similarity scores
        relevance_scores = F.softmax(similarity_scores, dim=-1)

        # Weight the input embeddings with relevance scores
        Y = X * relevance_scores.unsqueeze(-1)

        # Return weighted embeddings and updated Z
        return Y, {}


# WRITE YOUR UNIT TEST FUNCTIONS HERE #

@gau_test # DO NOT CHANGE THIS DECORATOR, OTHERWISE IT WON'T BE RECOGNIZED AS A UNIT TEST #
def test_relevance_scoring_gau(device=None, dtype=None)->None: # RENAME THIS FUNCTION, DO NOT CHANGE THE ARGUMENTS, IT SHOULD ALSO NOT RETURN ANYTHING #
    # AN AVAILABLE DEVICE AND DTYPE ARE PASSED AS ARGUMENTS, USE THEM TO INITIALIZE YOUR GAU AND MOCK INPUTS #

    # Initialize the GAU
    embed_dim = 512
    block_loc = (0, 6)
    relevance_scoring_gau = RelevanceScoringGAU(embed_dim=embed_dim, block_loc=block_loc, kwarg_all={}, device=device, dtype=dtype)

    # Mock input
    batch_size = 2
    seq_len = 10
    X = torch.randn(batch_size, seq_len, embed_dim, device=device, dtype=dtype)
    Z = {}

    # Forward pass
    Y, Z_ = relevance_scoring_gau(X, **Z)

    # Assertions
    assert Y.shape == (batch_size, seq_len, embed_dim), "Output shape mismatch"
    assert isinstance(Z_, dict), "Z_ should be a dictionary"

    # Print for debugging
    print("Output embeddings:", Y)
    print("Updated Z:", Z_)

    # Additional checks
    assert torch.allclose(Y.sum(dim=-1), X.sum(dim=-1), atol=1e-6), "Weighted embeddings should preserve the sum along the embedding dimension"

    print("RelevanceScoringGAU test passed!")
'''
children=['ProjectionGAU','SimilarityComputationGAU','NormalizationGAU']

In [64]:

unit_name = "XAU"  # Provide the unit_name to rename GAU class
reformatted_code, new_args, gau_tests, errors, warnings = check_and_reformat_gau_code(code, unit_name,children)
print("Reformatted Code:\n" + reformatted_code)
print("Errors:\n", errors)
print("Warnings:\n", warnings)
print("New Arguments:\n", new_args)


ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)
spec=P.UnitSpec(
   unitname = unit_name,
   document = 'Test',
   inputs = [],
   outputs = [],
)
test_tree.add_unit(
    spec, reformatted_code, new_args, None, None, None, children, gau_tests, None
)


AttributeError: 'Attribute' object has no attribute 'id'

In [57]:
code=test_tree.compose_unit('XAU')
code=U.add_line_num(code)
print(code)

line 1: import torch
line 2: import torch.nn as nn
line 3: from model_discovery.model.utils.modules import GAUBase, gau_test
line 4: import torch.nn.functional as F
line 5: 
line 6: 
line 7: class MemoryAccessUnit(nn.Module):
line 8: 
line 9:     def __init__(self, embed_dim, memory_size, device=None, dtype=None):
line 10:         super().__init__(embed_dim)
line 11: 
line 12:     def _forward(self, X, **Z):
line 13:         return X, {}
line 14: 
line 15: 
line 16: class DownsamplingUnit(nn.Module):
line 17: 
line 18:     def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
line 19:         super().__init__(embed_dim)
line 20: 
line 21:     def _forward(self, X, **Z):
line 22:         return X, {}
line 23: 
line 24: 
line 25: class XAU(GAUBase):
line 26:     """Generalized Autoregressive Block Unit
line 27:         Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}
line 28:         Output:       Y: (batch, seqlen, embed_d

In [61]:
code='''
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GAUBase, gau_test


class DilatedReformerRootGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.dilated_attention = DilatedAttentionGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.reversible_residual = ReversibleResidualGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **
            self.factory_kwargs, **kwarg_all)
        self.lsh_attention = LSHAttentionGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )

    def _forward(self, X, **Z):
        X, Z = self.dilated_attention(X, **Z)
        X, Z = self.reversible_residual(X, **Z)
        X, Z = self.lsh_attention(X, **Z)
        return X, Z


class LSHAttentionGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


class DilatedAttentionGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


class ReversibleResidualGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


@gau_test
def test_DilatedReformerRootGAU_unit_test_dilated_reformer_root_gau(device=
    None, dtype=None) ->None:
    batch_size = 2
    seq_len = 10
    embed_dim = 16
    X = torch.randn(batch_size, seq_len, embed_dim, device=device, dtype=dtype)
    Z = {}
    gau = DilatedReformerRootGAU(embed_dim=embed_dim, block_loc=(0, 6), kwarg_all={}, device=
        device, dtype=dtype)
    Y, Z_ = gau(X, **Z)
    assert Y.shape == X.shape, 'Output shape should match input shape'
    assert isinstance(Z_, dict), 'Z_ should be a dictionary'
    print('Input X:', X)
    print('Output Y:', Y)
    print('Intermediate variables Z_:', Z_)


def run_DilatedReformerRootGAU_tests():
	try:
		test_DilatedReformerRootGAU_unit_test_dilated_reformer_root_gau()
	except Exception as e:
		print("Error in running unit_test_dilated_reformer_root_gau:", e)


if __name__ == "__main__":
	run_DilatedReformerRootGAU_tests()
'''

import io
from contextlib import redirect_stdout

output=io.StringIO()
with redirect_stdout(output):
    exec(code)
captured = str(output.getvalue())

In [62]:
captured

'Input X: tensor([[[-2.3012e-01,  2.3788e+00, -4.7254e-01, -1.9962e+00,  8.4011e-01,\n          -1.3151e+00, -7.6405e-01, -4.4511e-01,  1.2063e+00, -7.1245e-01,\n          -4.3916e-01,  1.1784e+00, -1.7686e-01,  1.0315e-01,  1.3303e+00,\n           3.2917e-01],\n         [ 2.8818e-01, -1.5537e-01, -1.3801e+00, -1.4679e+00,  1.4772e+00,\n           1.6958e+00,  1.3572e+00, -4.7153e-01, -1.7442e+00, -6.8832e-01,\n           6.4387e-01, -1.7276e+00,  1.9131e+00, -2.7023e-01, -1.6691e+00,\n          -5.1265e-01],\n         [ 1.0191e+00, -5.3503e-01,  1.0546e+00,  3.6731e+00,  1.0722e+00,\n           1.7628e-02, -1.0685e+00, -2.2400e-01,  9.5643e-01,  1.6189e+00,\n           2.5520e-01,  8.3835e-01,  7.8880e-01,  3.5786e-02, -6.9103e-01,\n          -6.8120e-01],\n         [-8.3853e-01,  1.4766e-01, -1.7785e+00, -9.1734e-01,  2.0629e+00,\n           1.7541e-01, -1.0523e+00,  1.0929e+00,  1.2211e+00,  4.3829e-02,\n          -1.3583e+00,  6.1974e-01,  1.3561e+00, -8.1625e-01,  7.7795e-01,\n   

In [58]:

import io
from contextlib import redirect_stdout

output=io.StringIO()
with redirect_stdout(output):
    exec(test_tree.compose_unit('XAU'))

print(output.getvalue())

test1
test2



In [386]:
children={
    'unit1': 'MockUnit1',
    'unit2': 'MockUnit2',
}
called=['unit1', 'unit2', 'unit1']
test_tree.units['DualPathGAU'] = GAUNode('DualPathGAU', None, None, None, called, None, None, children, None)
children={
    'unit1': 'MockUnit3',
    'unit2': 'MockUnit4',
}
called=['unit2', 'unit2']
test_tree.units['MockUnit1'] = GAUNode('MockUnit1', None, None, None, called, None, None, children, None)

In [387]:
test_tree.units

{'DualPathGAU': GAUNode(name='DualPathGAU', code=None, args=None, desc=None, path=['unit1', 'unit2', 'unit1'], review=None, rating=None, children={'unit1': 'MockUnit1', 'unit2': 'MockUnit2'}, suggestions=None),
 'MockUnit1': GAUNode(name='MockUnit1', code=None, args=None, desc=None, path=['unit2', 'unit2'], review=None, rating=None, children={'unit1': 'MockUnit3', 'unit2': 'MockUnit4'}, suggestions=None)}

In [ ]:
class A:
    def __init__(self,r,d,e,**Z):
        print('Args in A:',d,e)
        print('Kwargs in A:',Z)

class B(A):
    def __init__(self,_Z,c=1,d=2,e=3,**Z):
        args2={'d':d,'e':e}
        super().__init__(c,**_Z,**args2)
        print('Kwargs in B:',Z)

cfg={
    'a':1,
    'b':2,
    'c':3,
}

B(cfg,**cfg)

Args in A: 2 3
Kwargs in A: {'a': 1, 'b': 2, 'c': 3}
Kwargs in B: {'a': 1, 'b': 2}


In [395]:
def A(X,Z,**kwargs):
    print(X)
    print(Z)
    print(kwargs)

def B(X,Z,a,**kwargs):
    A(X,Z,**Z)

Z={'a':3}
B(1,Z,**Z)

1
{'a': 3}
{'a': 3}


## Test


In [12]:
import os,sys
sys.path.append('..')
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GABBase


class GAB(GABBase):
    '''
    GAU Tree Map of Butterfly-AFT Generalized Autoregressive Block (BAGAB):
    ButterflyAFTGAU (Rating: 3.5/5)
        |- AFTMechanismGAU (Rating: 3.5/5)
            |- PositionBiasGAU (Rating: 3.5/5)
            |- ElementWiseOperationGAU (Rating: 3.5/5)
        |- ButterflyTransformGAU (Rating: 3.5/5)
            |- ButterflyLayerGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
            |- ButterflyInitializationGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
                |- ButterflyParameterGAU (Unimplemented)

    Implemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ElementWiseOperationGAU, ButterflyAFTGAU, ButterflyInitializationGAU, ButterflyStageGAU
    Unimplemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ButterflyParameterGAU, ElementWiseOperationGAU, ButterflyInitializationGAU, ButterflyStageGAU
    '''

    def __init__(self, embed_dim: int, block_loc: tuple, device=None, dtype
        =None, **kwargs):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc)
        self.root = ButterflyAFTGAU(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwargs, **factory_kwargs, **kwargs)

    def _forward(self, X, **Z):
        X, Z = self.root(X, **Z)
        return X, Z


from model_discovery.model.utils.modules import GAUBase


class ButterflyAFTGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.aft_mechanism = AFTMechanismGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.butterfly_transform = ButterflyTransformGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        aft_output, Z = self.aft_mechanism(X, **Z)
        butterfly_output, Z = self.butterfly_transform(aft_output, **Z)
        return butterfly_output, Z


class ButterflyTransformGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_layer = ButterflyLayerGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_init = ButterflyInitializationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.butterfly_init(X, **Z)
        Y, Z = self.butterfly_layer(X, **Z)
        return Y, Z


class ButterflyLayerGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.stages = nn.ModuleList([ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all) for _ in range(kwarg_all.get('num_stages', 4))])

    def _forward(self, X, **Z):
        for stage in self.stages:
            X, Z = stage(X, **Z)
        return X, Z

class ButterflyMatrixGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


import torch.nn.functional as F


class ButterflyInitializationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_size = kwarg_all.get('butterfly_size', embed_dim)
        self.butterfly_matrices = nn.ParameterList([nn.Parameter(torch.
            randn(self.butterfly_size, self.butterfly_size, **self.
            factory_kwargs)) for _ in range(int(torch.log2(torch.tensor(
            self.butterfly_size))))])
        self.butterfly_stage = ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_parameter = ButterflyParameterGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        for matrix in self.butterfly_matrices:
            X = F.linear(X, matrix)
        return X, Z


class ButterflyParameterGAU(GAUBase):
    """
    Generalized Autoregressive Block Unit for learning parameters of Butterfly Factorization.
    Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
    Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
    Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

    embed_dim:    The dimension of the input embeddings
    block_loc:    The location of the block within the network, (layer_idx, n_block)
    kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.param_size = kwarg_all.get('param_size', 128)
        self.butterfly_weights = nn.Parameter(torch.randn(self.param_size,
            embed_dim, **self.factory_kwargs))
        nn.init.xavier_uniform_(self.butterfly_weights)

    def _forward(self, X, **Z):
        transformed_X = torch.matmul(X, self.butterfly_weights)
        Z_ = {'butterfly_weights': self.butterfly_weights}
        return transformed_X, Z_
        # transformed_X = F.linear(X, self.butterfly_weights.T)
        # return transformed_X, {}


class ButterflyComputationGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


class ButterflyStageGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_params = ButterflyParameterGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)

    def _forward(self, X, **Z):
        transformed_X, Z = self.butterfly_params(X, **Z)
        assert transformed_X.shape == X.shape, f'Output shape {transformed_X.shape} does not match input shape {X.shape}'
        return transformed_X, Z

class AFTMechanismGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = PositionBiasGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.element_wise_operation = ElementWiseOperationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.position_bias(X, **Z)
        Y, Z = self.element_wise_operation(X, **Z)
        return Y, Z


class ElementWiseOperationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, embed_dim, **self.
            factory_kwargs))

    def _forward(self, X, **Z):
        queries = Z.get('queries', X)
        keys = Z.get('keys', X)
        values = Z.get('values', X)
        elementwise_product = queries * keys
        elementwise_sum = elementwise_product + values + self.position_bias
        return elementwise_sum, {}


class PositionBiasGAU(GAUBase):
    """Position Bias Generalized Autoregressive Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, 1, embed_dim, **
            self.factory_kwargs))

    def _forward(self, X, **Z):
        Y = X + self.position_bias
        return Y, {}
    

class GABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100, **kwargs):
        super().__init__()
        self.gab = GAB(embed_dim, (0,1), **kwargs)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, Z = self.gab(X, **Z)
        return self.lm_head(X),X



gab_config = {}


In [13]:
class SimpleGABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100):
        super().__init__()
        self.butterfly_param = ButterflyParameterGAU(embed_dim, (0,1), {})
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, _ = self.butterfly_param(X, **Z)
        return self.lm_head(X), X

# Test the simplified model
simple_model = GABLM(128)
mock_input = torch.randn(1, 10, 128)
mock_y = torch.randint(0, 100, (1, 10))

optimizer = optim.Adam(simple_model.parameters())
criterion = nn.CrossEntropyLoss()

# Training loop
for _ in range(5):  # Run for a few iterations
    optimizer.zero_grad()
    y, X = simple_model(mock_input)
    loss = criterion(y.view(-1, 100), mock_y.view(-1))
    loss.backward()
    print(f"Loss: {loss.item()}")
    for name, param in simple_model.named_parameters():
        if param.grad is not None:
            print(f"Gradient for {name}: {param.grad.abs().mean()}")
        else:
            print(f"No gradient for {name}")
    optimizer.step()

Loss: 54495796.0
Gradient for gab.root.aft_mechanism.position_bias.position_bias: 899675.4375
Gradient for gab.root.aft_mechanism.element_wise_operation.position_bias: 475023.78125
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.0.butterfly_params.butterfly_weights: 810216.875
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.1.butterfly_params.butterfly_weights: 777344.75
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.2.butterfly_params.butterfly_weights: 783032.0625
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.3.butterfly_params.butterfly_weights: 771978.3125
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.0: 62701.796875
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.1: 66148.203125
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.2: 67731.2734375
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.3: 69984.

In [9]:
from torch import nn, optim 

model=GABLM(128, **gab_config)
vocab_size=100
DEFAULT_CONTEXT_LENGTH=10

mock_input=torch.randn(1, DEFAULT_CONTEXT_LENGTH, 128)
mock_y=torch.randint(0, 100, (1, 10))
mock_Z={}


criterion = nn.CrossEntropyLoss()
model.train()
optimizer = optim.Adam(model.parameters())

# Zero the parameter gradients
optimizer.zero_grad()
y,X=model(mock_input, **mock_Z)
loss=nn.CrossEntropyLoss()(y.view(-1, 100), mock_y.view(-1))
loss.backward()

used_params = set()

# Hook to register which parameters are used during the forward pass
def hook_fn(module, input, output):
    for param in module.parameters():
        used_params.add(param)

# Register hooks for all submodules
hooks = []
for submodule in model.modules():
    hook = submodule.register_forward_hook(hook_fn)
    hooks.append(hook)

# Forward pass again to collect used parameters
model(mock_input)

# Remove all hooks
for hook in hooks:
    hook.remove()

all_gradients_present = True

# Check gradients
for name, param in model.named_parameters():
    if param.requires_grad:
        if param in used_params and param.grad is None:
            # Used parameter without gradient -> Error
            print(f"Error: Used parameter {name} requires gradients but has none.")
            all_gradients_present = False
        elif param not in used_params:
            # Unused parameter -> Warning
            print(f"Warning: Parameter {name} was not used in the forward pass.")

if all_gradients_present:
    print('Differentiability test passed')
else:
    print('Differentiability test failed due to missing gradients.')


Error: Used parameter gab.root.butterfly_transform.butterfly_init.butterfly_stage.butterfly_params.butterfly_weights requires gradients but has none.
Error: Used parameter gab.root.butterfly_transform.butterfly_init.butterfly_parameter.butterfly_weights requires gradients but has none.
Differentiability test failed due to missing gradients.
